In [1]:
pip install transformers datasets torch accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
pip install peft

In [3]:
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
from transformers import AdamW
from peft import LoraConfig, get_peft_model

In [4]:
df = pd.read_excel('/content/final_clinical_mutation_data.xlsx')

In [5]:
df = df.head(10000)

In [6]:
# Preprocess the dataset
df['text'] = df[['Hugo_Symbol', 'Chromosome', 'Consequence', 'Variant_Type', 'Reference_Allele', 'Tumor_Seq_Allele1', 'Tumor_Seq_Allele2']].astype(str).agg(' '.join, axis=1)
df = df[['text', 'Variant_Classification']]


In [7]:
# Encode labels
label_mapping = {label: idx for idx, label in enumerate(df['Variant_Classification'].unique())}
df['label'] = df['Variant_Classification'].map(label_mapping)

In [8]:
# Split into train and test
train_df = df.sample(frac=0.8, random_state=42)
test_df = df.drop(train_df.index)

In [9]:
# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df[['text', 'label']])
test_dataset = Dataset.from_pandas(test_df[['text', 'label']])
dataset = DatasetDict({"train": train_dataset, "test": test_dataset})

In [10]:
# Load tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(label_mapping))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
 #Tokenize data
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=128)

In [12]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [13]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
# Fully Fine-tuning
def train_model(tokenized_datasets, model, training_args):
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
        tokenizer=tokenizer,
        compute_metrics=lambda p: {"accuracy": (p.predictions.argmax(-1) == p.label_ids).astype(float).mean()}
    )
    trainer.train()
    return trainer.evaluate()

In [15]:
# LoRa Fine-tuning
def train_model_lora(tokenized_datasets, model, training_args):
    lora_config = LoraConfig(
        r=8,
        lora_alpha=32,
        lora_dropout=0.1,
        target_modules=["query", "key"],
        bias="none",
        task_type="SEQ_CLS"
    )
    lora_model = get_peft_model(model, lora_config)
    trainer = Trainer(
        model=lora_model,
        args=training_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
        tokenizer=tokenizer,
        compute_metrics=lambda p: {"accuracy": (p.predictions.argmax(-1) == p.label_ids).astype(float).mean()}
    )
    trainer.train()
    return trainer.evaluate()

In [20]:
# Layer Freezing Fine-tuning
def train_model_layer_freezing(tokenized_datasets, model, training_args):
    for param in model.roberta.embeddings.parameters():
        param.requires_grad = True
    for layer in model.roberta.encoder.layer[:-6]:
        for param in layer.parameters():
            param.requires_grad = False

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
        tokenizer=tokenizer,
        compute_metrics=lambda p: {"accuracy": (p.predictions.argmax(-1) == p.label_ids).astype(float).mean()}
    )
    trainer.train()
    return trainer.evaluate()


In [17]:

results_fully_finetuned = train_model(tokenized_datasets, model, training_args)


<ipython-input-14-eb1ec4aac905>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,0.060100,0.037810,0.989000
2,0.027100,0.021287,0.991000
3,0.005600,0.011240,0.997500


In [18]:
results_lora = train_model_lora(tokenized_datasets, model, training_args)

<ipython-input-15-e578eb7ad04e>:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.008800,0.012082,0.996500
2,0.003500,0.010510,0.997500
3,0.002900,0.010169,0.998000


In [21]:
results_layer_freezing = train_model_layer_freezing(tokenized_datasets, model, training_args)

<ipython-input-20-cddcb469e8db>:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.006500,0.012275,0.997000
2,0.001600,0.010314,0.998500
3,0.001700,0.010023,0.998500


In [22]:

print("Fully Fine-tuned Model Results:", results_fully_finetuned)
print("LoRa Fine-tuned Model Results:", results_lora)
print("Layer Freezing Fine-tuned Model Results:", results_layer_freezing)

Fully Fine-tuned Model Results: {'eval_loss': 0.01124021876603365, 'eval_accuracy': 0.9975, 'eval_runtime': 4.5929, 'eval_samples_per_second': 435.453, 'eval_steps_per_second': 54.432, 'epoch': 3.0}
LoRa Fine-tuned Model Results: {'eval_loss': 0.010169061832129955, 'eval_accuracy': 0.998, 'eval_runtime': 5.0043, 'eval_samples_per_second': 399.659, 'eval_steps_per_second': 49.957, 'epoch': 3.0}
Layer Freezing Fine-tuned Model Results: {'eval_loss': 0.010023231618106365, 'eval_accuracy': 0.9985, 'eval_runtime': 4.8339, 'eval_samples_per_second': 413.745, 'eval_steps_per_second': 51.718, 'epoch': 3.0}
